### 1. MovieData Relational Operations

In [1]:
import os

from datetime import datetime, timedelta
from omdb import OMDBClient
client = OMDBClient(apikey='e55a0e19')

import pandas as pd

import datetime

# define input and output folder paths
input_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\input_files'
output_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\output_files'

relational_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\input_files\relational'
# staging_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\input_files\staging'

user_created_data_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\input_files\user_created_moviedata'
staging_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\output_files\4_staging'
raw_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\output_files\3_raw'
curated_folder = r'C:\My\Workspace\Python_Projects\Movies_Data\output_files\2_curated'

In [2]:
# 7/16/23 copying entire block from Movie_Data_Relational_Nb_dev1a, but it still needs to be double-checked to eliminate functions not used in this notebook

# helper function to search omdb for each film and return the unique imdb ID
def get_film_object(row):
    film_title = row['Title']
    film_year_released = str(row['Year_Released'])

    try:
        film_object = client.search(film_title, year=film_year_released)
        return film_object[0]
    except Exception as e:
        print(f"Skipping row: {row} - Error: {e}")
        return None

# helper function to extract the unique imdb ids from the list
def extract_column_to_list(df, column_name):
    return df[column_name].tolist()

# helper function to create lists of tuples for imdb key and other dictionary keys
def create_data_structure(item, key_name):
    values = item[key_name].split(', ')
    imdb_id = item['imdbID']
    tuple_result = [(imdb_id, value) for value in values]
    return tuple_result

# helper function to flatten each ragged list
def flatten_data_structure(main_list):
    flattened_list = []
    for sublist in main_list:
        for item in sublist:
            flattened_list.append(item)
    return flattened_list

# helper function to create lookup table
def return_lookup_dataframe(ph_data_list, ph_key):
    # Step 1: Create data structure
    cat_obj = [create_data_structure(ph_dl, ph_key) for ph_dl in ph_data_list]
    # Step 2: Flatten the data structure
    ph_flattened_list = flatten_data_structure(cat_obj)
    # Step 3: Create the lookup DataFrame
    ph_lookup_df = pd.DataFrame(ph_flattened_list, columns=column_set)
    # Step 4: Add key id column
    ph_lookup_df['key'] = range(1, len(ph_lookup_df) + 1)
    # Step 6: Add 'date_updated' column with current date
    ph_lookup_df['date_updated'] = datetime.datetime.now().date()
    # Step 5: Reorder the columns
    ph_lookup_df = ph_lookup_df[['key', column_set[0], column_set[1], 'date_updated']]
    # Step 6: Return dataframe
    return ph_lookup_df

    # this code block may serve a better purpose by returning a dataframe
def return_distinct_dated_dataframe(ph_data_list, ph_key):
    # Step 1: Create data structure
    cat_obj = [create_data_structure(ph_dl, ph_key) for ph_dl in ph_data_list]
    # Step 2: Flatten the data structure
    ph_flattened_list = flatten_data_structure(cat_obj)
    # Step 3: Create the distinct Dimension DataFrame
    ph_dimension_df = pd.DataFrame(ph_flattened_list, columns=column_set)
    ph_dimension_df.drop(columns=column_set[0], inplace=True)
    # Step 4: Drop duplicates before adding key column
    ph_dimension_df.drop_duplicates(subset=column_set[1], inplace=True)
    # Step 5: Add id column
    ph_dimension_df['id'] = range(1, len(ph_dimension_df) + 1)
    # Step 6: Add 'date_updated' column with current date
    ph_dimension_df['date_updated'] = datetime.datetime.now().date()
    # Step 7: Reorder the columns
    ph_dimension_df = ph_dimension_df[['id', column_set[1], 'date_updated']]
     # Step 8: Return Dataframe
    return ph_dimension_df

##### 3. Import pickle file and load into dataframe

In [3]:
# this reads a file that was created in Movies_Data_fromAPItoExport notebook
import pickle

# movie_data_list = []
# the above empty list declaration can be removed if no side effects

file_name = 'data_list.pkl'
output_file = os.path.join(staging_folder, file_name)
with open(output_file, 'rb') as fp:
    movie_data_list = pickle.load(fp)

In [8]:
# 7/21/23 save this task for later
# import inflation_table into lookup dataframe
# create path and name variable
file_name = 'inflation_table.csv'
input_file = os.path.join(input_folder, file_name)
# read csv
inflation_df = pd.read_csv(input_file, delimiter='|')
inflation_df.columns

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 1109: invalid start byte

In [ ]:
# persist inflation dataframe
# inflation_df.to_csv(output_file, sep='|', index=False)

##### 4. Clean and Transform dataframe while adding key and re-ordering

In [15]:
# create 3NF film table
#make dataframe with minimal columns
# movie_data_df is made using the full movie_data_list
movie_data_df = pd.DataFrame(movie_data_list)
movie_data_df.columns

Index(['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director',
       'Writer', 'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Poster',
       'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID', 'Type',
       'DVD', 'BoxOffice', 'Production', 'Website', 'Response',
       'InternetMovieDatabase', 'RottenTomatoes', 'Metacritic'],
      dtype='object')

In [16]:
# experiment to check types
"""
trial_df = movie_data_df.copy()
trial_df['BoxOffice'] = trial_df['BoxOffice'].replace('N/A','$0')
# trial_df.dropna(subset=['BoxOffice'], inplace=True)
trial_df['BoxOffice'].apply(type).value_counts()
"""

<class 'str'>    71
Name: BoxOffice, dtype: int64

In [ ]:
# experiment to check types
"""
trial_df['BoxOffice'] = trial_df['BoxOffice'].str.strip()
trial_df['BoxOffice'] = trial_df['BoxOffice'].str.replace(',','').str.replace('$','').astype('int')
trial_df['BoxOffice'].apply(type).value_counts()
"""

In [4]:
# add id column
movie_data_df['id'] = range(1, len(movie_data_df) + 1)
# choose columns
column_list=["id","imdbID","Title","Year","Rated","Released","Runtime","Plot","Poster","Metascore","Metacritic","InternetMovieDatabase","RottenTomatoes","imdbRating","imdbVotes","Type","DVD","BoxOffice"]
# creating a narrow data frame just for films in the list
film_data_df = pd.DataFrame(movie_data_df, columns=column_list)

In [5]:
# Replace NaN values in specific columns of movie_data_df
columns_to_fill = {
    "Released": "1/1/1900",
    "DVD": "1/1/1900",
    "Year": "1900",
    "RottenTomatoes": "0%",
    "BoxOffice": "$0",
    "Metascore": "0",
    "imdbRating": "0",
    "imdbVotes": "0"
}

na_value = 'N/A'

for column, value in columns_to_fill.items():
    film_data_df[column] = film_data_df[column].replace(na_value ,value)
    # film_data_df[column] = film_data_df[column].fillna(value)

In [6]:
file_name = 'films.csv'
output_file = os.path.join(relational_folder, file_name)
film_data_df.to_csv(output_file, sep='|', index=False)

##### 5. Create master, relational, & lookup dataframes and exports

In [7]:
lookup_data = [
    {
        'file_name': 'film_actors.csv',
        '3NF_file_name': 'actors.csv',
        'column_set': ['imdb_id', 'actors'],
        'table_name': 'Actors'
    },
    {
        'file_name': 'film_directors.csv',
        '3NF_file_name': 'directors.csv',
        'column_set': ['imdb_id', 'director'],
        'table_name': 'Director'
    },
    {
        'file_name': 'film_writers.csv',
        '3NF_file_name': 'writers.csv',
        'column_set': ['imdb_id', 'writer'],
        'table_name': 'Writer'
    },
    {
        'file_name': 'film_genres.csv',
        '3NF_file_name': 'genres.csv',
        'column_set': ['imdb_id', 'genre'],
        'table_name': 'Genre'
    },
    {
        'file_name': 'film_languages.csv',
        '3NF_file_name': 'languages.csv',
        'column_set': ['imdb_id', 'language'],
        'table_name': 'Language'
    },
    {
        'file_name': 'film_countries.csv',
        '3NF_file_name': 'countries.csv',
        'column_set': ['imdb_id', 'country'],
        'table_name': 'Country'
    },
    {
        'file_name': 'film_titles.csv',
        '3NF_file_name': 'titles.csv',
        'column_set': ['imdb_id', 'title'],
        'table_name': 'Title'
    }
]
# the following for loop should include just returning a dataframe instead of making a table for a file export if it accomplishes a goal. The file exports may still be need outside the operational application
for data in lookup_data:
    file_name = data['file_name']
    rel_file_name = data['3NF_file_name']
    output_file = os.path.join(relational_folder, file_name)
    column_set = data['column_set']
    table_name = data['table_name']

    lookup_df = return_lookup_dataframe(movie_data_list, table_name)
    master_df = return_distinct_dated_dataframe(movie_data_list, table_name)
    print("stop")

    # Joining master_data_df, lookup_df, and film_data_df
    reference_lookup = master_df.merge(lookup_df, on=master_df.columns[1]).merge(film_data_df, left_on='imdb_id', right_on='imdbID')

    # Selecting the desired columns
    # reference_lookup = reference_lookup[['id_x', 'id_y']].rename(columns={'id_x': 'm.id', 'id_y': 'f.id'})
    reference_lookup = reference_lookup[['id_y', 'id_x', 'date_updated_y']].rename(columns={'id_x': 'm.id', 'id_y': 'f.id'})
    reference_lookup.sort_values("f.id")


    # Printing the resulting DataFrame
    print(reference_lookup)

    output_file = os.path.join(relational_folder, file_name)
    reference_lookup.to_csv(output_file, sep='|', index=False)
    output_file = os.path.join(relational_folder, rel_file_name)
    master_df.to_csv(output_file, sep='|', index=False)

stop
     f.id  m.id date_updated_y
0       1     1     2023-07-16
1       1     2     2023-07-16
2       1     3     2023-07-16
3       2     4     2023-07-16
4       2     5     2023-07-16
..    ...   ...            ...
208    68   141     2023-07-16
209    68   146     2023-07-16
210    65   141     2023-07-16
211    65   142     2023-07-16
212    65   143     2023-07-16

[213 rows x 3 columns]
stop
    f.id  m.id date_updated_y
0      1     1     2023-07-16
1      2     2     2023-07-16
2      3     2     2023-07-16
3      4     3     2023-07-16
4     64     3     2023-07-16
..   ...   ...            ...
80    62    57     2023-07-16
81    63    58     2023-07-16
82    69    59     2023-07-16
83    70    60     2023-07-16
84    71    61     2023-07-16

[85 rows x 3 columns]
stop
     f.id  m.id date_updated_y
0       1     1     2023-07-16
1       1     2     2023-07-16
2       2     3     2023-07-16
3       2     4     2023-07-16
4       2     5     2023-07-16
..    ...   ...     